In [ ]:
#r "nuget: FSharp.Stats, 0.4.3"
#r "nuget: BioFSharp, 1.2.0"
#r "nuget: BioFSharp.IO, 1.2.0"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Deedle, 2.5.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: ARCtrl"
#r "nuget: ARCtrl.NET, 1.0.5"
#r "nuget: ARCtrl.QueryModel, 1.0.5"

open System.IO
open FSharp.Stats
open BioFSharp
open Plotly.NET
open Deedle
open FSharpAux
open ARCtrl
open ARCtrl.NET
open ARCtrl.QueryModel
open ARCtrl.ISA


# NB08c Absolute Quantification (for SDS-PAGE results)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CSBiology/BIO-BTE-06-L-7/gh-pages?filepath=NB06d_Absolute_Quantification_SDS.ipynb)

[Download Notebook](https://github.com/CSBiology/BIO-BTE-06-L-7/releases/download/NB08c/NB08c_Absolute_Quantification_SDS.ipynb)

Finally, after careful peptide ion selection, quality control and assuring that our label efficiency allows accurate for quantifications, we can start to
calculate protein abundancies. Since we start again by getting access to our data and its description, this notebook will start off familiar!

## I. Reading the sample description

As always: before we analyze our data, we will download and read the sample description provided by the experimentalist.


In [ ]:
let path = @"..\"

let arc = ARC.load path
let i = arc.ISA.Value

Next, we will prepare functions to look up parameters which might be needed for further calculations.
If you compare this list to the one of note book NB06b you will find additional functions. We will need these functions
in order to calculate the absolute abundances. 


In [ ]:
type CutoutBand =
    | RbcL
    | RbcS

let normalizeFileName (f : string) = if Path.HasExtension f then f else Path.ChangeExtension(f, "wiff")

//
let getStrain (fileName : string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("strain")
        .[0]
        .ValueText

//
let getExpressionLevel (fileName : string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("gene expression")
        .[0]
        .ValueText

// 
let getμgChlorophPerMlCult (fileName: string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("total chlorophyll concentration of culture#7")
        .[0]
        .ValueText
        |> float

// 
let getCellCountPerMlCult (fileName: string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("cell concentration#6")
        .[0]
        .ValueText
        |> float

// 
let getμgChlorophPerμlSample (fileName: string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("total chlorophyll of sample#12")
        .[0]
        .ValueText
        |> float
        |> fun x -> x / 1000.
// 
let getμgProtPerμlSample (fileName: string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Cultivation")
        .WithName("whole cell protein concentration of sample#11")
        .[0]
        .ValueText
        |> float
        |> fun x -> x / 1000.

//
let get15N_PS_Amount (fileName : string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.ParametersOf(fN,"Protein extraction SDS")
        .WithName("15N Photosynthesis QconCAT mass")
        .[0]
        .ValueText
//
let getGroupID (fileName : string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.ParametersOf(fN,"Protein extraction SDS")
        .WithName("Group name")
        .[0]
        .ValueText

let getLoadAmount (fileName : string) =
    let fN = fileName |> normalizeFileName
    i.ArcTables.CharacteristicsOf(fN,"Sample preparation (PAGE)")
        .WithName("soluble protein content")
        .[0]
        .ValueText
        |> float

let getCutoutBand (fileName : string) =
    let fN = fileName |> normalizeFileName
    match fN.Contains("rbcL"), fN.Contains("rbcS") with
    | true,false -> RbcL
    | false,true -> RbcS
    | _ -> failwith (sprintf "rbcL or rbcS not cut out in file %s" fN)


A quick execution to test the retrieval of data from the isa sample table:


In [ ]:
getStrain "Gr2rbcL2.wiff"
getExpressionLevel "Gr2rbcL2.wiff"
getμgChlorophPerMlCult "Gr2rbcL2.wiff"
getCellCountPerMlCult "Gr2rbcL2.wiff"
getμgChlorophPerμlSample "Gr2rbcL2.wiff"
getμgProtPerμlSample "Gr2rbcL2.wiff"
get15N_PS_Amount "Gr2rbcL2.wiff"
getGroupID "Gr2rbcL2.wiff"
getLoadAmount "Gr2rbcL2.wiff"


## II. Reading the data
As promised, we start this notebook with the output of the previous analysis, this notebook assumes that the data from *NB06b Data Access and Quality Control* is stored in a .txt


In [ ]:
type PeptideIon = 
    {|
        ProteinGroup    : string  
        Synonyms        : string
        StringSequence  : string
        PepSequenceID   : int
        Charge          : int
    |}

//This is the filepath you chose in *NB08a Data Access and Quality Control (results for SDS-PAGE)*
let filePath = @"Your\Path\Here"

let qConcatDataFiltered =
    Frame.ReadCsv(path = filePath, separators = "\t")
    // StringSequence is the peptide sequence
    |> Frame.indexRowsUsing (fun os -> 
        {|
            ProteinGroup    = os.GetAs<string>("ProteinGroup"); 
            Synonyms        = os.GetAs<string>("Synonym")
            StringSequence  = os.GetAs<string>("StringSequence");
            PepSequenceID   = os.GetAs<int>("PepSequenceID");
            Charge          = os.GetAs<int>("Charge");
        |}
    )
    |> Frame.filterRows (fun k s -> String.contains "QProt_newPS" k.ProteinGroup)

let inline printIndexedData (f: Frame<{| Charge: int; PepSequenceID: int; ProteinGroup: string; StringSequence: string; Synonyms: string |},string>) =
    f
    |> Frame.mapRowKeys (fun k -> $"{k.ProteinGroup},{k.Synonyms},{k.StringSequence},{k.PepSequenceID},{k.Charge}")
    |> fun f -> f.Print()


## III. From Ratios to mol proteins per cell.

Now we can use the extensive information stored in the sample sheet and map each quantified peptide ion passing
the quality checks to an estimator for protein abundance! First we start off by defining a function to extract ratios:


In [ ]:
let sliceQuantColumns quantColID frame = 
    frame
    |> Frame.filterCols (fun ck os -> ck |> String.contains ("." + quantColID))
    |> Frame.mapColKeys (fun ck -> ck.Split('.') |> Array.item 0)


Next up, we have to define a function, which maps the measured ratio and measured parameters to an quantification value.


In [ ]:
/// 
let calcAbsoluteAbundance μgChlorophPerMlCult cellCountPerMlCult μgChlorophPerμlSample μgProtPerμlSample μgQProtSpike μgloadedProtein molWeightQProt molWeightTargetProt ratio1415N =
    let chlorophPerCell : float = μgChlorophPerMlCult / cellCountPerMlCult
    let cellsPerμlSample = μgChlorophPerμlSample / chlorophPerCell
    let μgProteinPerCell = μgProtPerμlSample / cellsPerμlSample
    let molQProtSpike = μgQProtSpike * 10. ** -6. / molWeightQProt
    let molProtPerBand = ratio1415N * molQProtSpike
    let molProtIn1μgLoadedProt = molProtPerBand / μgloadedProtein
    let gTargetProtIn1μgLoadedProt = molWeightTargetProt * molProtIn1μgLoadedProt
    let molProteinPerCell = molProtIn1μgLoadedProt * μgProteinPerCell
    let proteinsPerCell = molProteinPerCell * 6.022 * 10. ** 23.
    let attoMolProteinPerCell = molProteinPerCell * (10. ** 18.)
    {|
        MassTargetProteinInLoadedProtein    = gTargetProtIn1μgLoadedProt
        ProteinsPerCell                     = proteinsPerCell
        AttoMolProteinPerCell               = attoMolProteinPerCell
    |}


Inspecting the input parameters of 'calcAbsoluteAbundance' we can see that we need both, the molcular weight of the qProtein and of the 
native Protein. Since we have none at hand we will use our newly aquired skills to compute both and add them to the row key of our Frame. 


In [ ]:
/// Fasta item contains header and sequence
type FastaItem<'a> = {
        Header    : string;
        Sequence  : 'a;       
    }

/// Creates with header line and sequence.
let createFastaItem header sequence =
    { Header = header; Sequence = sequence }

// Conditon of grouping lines
let private same_group l =             
    not (String.length l = 0 || l.[0] <> '>')

/// Reads FastaItem from file. Converter determines type of sequence by converting seq<char> -> type
let fromFileEnumerator (converter:seq<char>-> 'a) (fileEnumerator) =
    // Matches grouped lines and concatenates them
    let record d (converter:seq<char>-> 'a) = 
        match d with
        | [] -> raise (System.Exception "Incorrect FASTA format")
        | (h:string) :: t when h.StartsWith ">" ->  let header = h .Remove(0,1)
                                                    let sequence = (Seq.concat t) |> converter
                                                    createFastaItem header sequence
                                                    
        | h :: _ -> raise (System.Exception "Incorrect FASTA format")        

    // main
    fileEnumerator
    |> Seq.filter (fun (l:string) -> not (l.StartsWith ";" || l.StartsWith "#"))
    |> Seq.groupWhen same_group 
    |> Seq.map (fun l -> record (List.ofSeq l) converter)


/// Reads FastaItem from file. Converter determines type of sequence by converting seq<char> -> type
let fromFile converter (filePath) =
    FSharpAux.IO.FileIO.readFile filePath
    |> fromFileEnumerator converter

In [ ]:
let path = @"..\studies\ProteinPreparation\resources\Chlamy_JGI5_5_Cp_Mp_QProt.fasta"

let examplePeptides = 
    path
    |> fromFile BioArray.ofAminoAcidString
    |> Array.ofSeq


First we find the sequences of the qProteins, calculate their masses and define a function to retrieve the calculated mass.


In [ ]:
let PS = 
    examplePeptides 
    |> Seq.find (fun prot -> prot.Header |> String.contains "QProt_newPS")

let PSMass = 
    BioFSharp.BioSeq.toMonoisotopicMassWith (Formula.monoisoMass Formula.Table.H2O) PS.Sequence


Then we repeat the process and assign the calculated masses to each protein.


In [ ]:
let withProteinWeights = 
    qConcatDataFiltered
    /// For each row (peptide) in the frame...
    |> Frame.mapRowKeys (fun k -> 
        let proteinsOfInterest = 
            k.ProteinGroup 
            |> String.split ';' 
            |> Array.filter (fun x -> x.Contains "Cre")
        let masses = 
            proteinsOfInterest
            /// ...we look up the matching protein sequence
            |> Seq.choose (fun creID -> 
                examplePeptides 
                |> Seq.tryFind (fun prot -> prot.Header |> String.contains creID)
                )
            /// ... and calculate the protein masses        
            |> Seq.map (fun prot -> 
                BioFSharp.BioSeq.toMonoisotopicMassWith (Formula.monoisoMass Formula.Table.H2O) prot.Sequence 
                )
        let avgMass = if Seq.isEmpty masses then 0. else masses |> Seq.average
        /// ... and add the average to the peptide.   
        {|k with AverageProtGroupMass = avgMass|}
    )
    
let inline printIndexedWeightData (f: Frame<{| AverageProtGroupMass: float; Charge: int; PepSequenceID: int; ProteinGroup: string; StringSequence: string; Synonyms: string |},string>) =
    f
    |> Frame.mapRowKeys (fun k -> $"{k.ProteinGroup},{k.Synonyms},{k.StringSequence},{k.PepSequenceID},{k.Charge},{k.AverageProtGroupMass}")
    |> fun f -> f.Print()

With our newest update to our meta data (adding the masses to the rowkey), we can slice out the columns
needed to calculate absolute abundances: the ratio columns.


In [ ]:
let ratios = sliceQuantColumns "Ratio" withProteinWeights


In [ ]:
ratios
|> printIndexedWeightData


Finally, we can iterate the ratios and map each to a protein abundance using our well annotated experiment.


In [ ]:
//
let absoluteAbundances = 
    ratios
    |> Frame.map (fun peptide fileName ratio -> 
        try 
            let μgChlorophPerMlCult     = getμgChlorophPerMlCult fileName
            let cellCountPerMlCult      = getCellCountPerMlCult fileName
            let μgChlorophPerμlSample   = getμgChlorophPerμlSample fileName
            let μgProtPerμlSample       = getμgProtPerμlSample fileName
            let μgQProtSpike            = get15N_PS_Amount fileName
            let μgloadedProtein         = getLoadAmount fileName
            let molWeightQProt          = PSMass
            let molWeightTargetProt     = peptide.AverageProtGroupMass
            let result = 
                calcAbsoluteAbundance
                    μgChlorophPerMlCult  
                    cellCountPerMlCult   
                    μgChlorophPerμlSample
                    μgProtPerμlSample    
                    (float μgQProtSpike)         
                    μgloadedProtein
                    molWeightQProt       
                    molWeightTargetProt
                    ratio
            result.AttoMolProteinPerCell
        with :? System.FormatException -> nan
    )


In [ ]:
absoluteAbundances
|> printIndexedWeightData

// Why don't we see results for the SDS experiments with CBB-QProt?
// Why don't we see any results in the BN experiment?
// Discuss (and maybe look into your assay file if you don't progress).


To see if our calculations are not off, we look at the calculated abundance for the well studied abundances of rbcL and RBCS
and compare this to the published knowledge about these proteins.
For this, we write a function that, given a protein synonym and a list of peptide sequences, returns a list of quantifications (via mean)
and the estimated uncertainty (via standard deviation). The results can then be visualized using e.g. column charts.


In [ ]:
let extractAbsoluteAbundancesOf filterCutoutProtein groupID prot peptidelist = 
    absoluteAbundances
    |> Frame.filterRows (fun k s -> k.Synonyms |> String.contains prot)
    |> Frame.filterRows (fun k s -> 
        peptidelist |> List.exists (fun (sequence,charge) -> sequence = k.StringSequence && charge = k.Charge)
    )
    |> Frame.getNumericCols 
    |> Series.observationsAll
    |> Seq.sortBy (fun (k, x) -> getLoadAmount k)
    |> Seq.filter (fun (k, x) -> getGroupID k = groupID)
    // why do we filter out the protein which is not cut out here?
    |> fun res ->
        if filterCutoutProtein then
            res
            |> Seq.filter (fun (k, x) -> 
                match getCutoutBand k with
                | RbcL -> String.contains (prot.ToLower()) (k.ToLower())
                | RbcS -> String.contains (prot.ToLower()) (k.ToLower())
            )
        else res
    |> Seq.choose (fun (k,x) -> 
        match x with
        | Some x -> Some (k, x)
        | _ -> None
    )
    |> Seq.map (fun (k, v) -> 
        {|
            Filename   = k 
            MeanQuant  = Stats.mean v
            StdevQuant = Stats.stdDev v
        |}
    )

// with filtering
let rbclQuantification = 
    extractAbsoluteAbundancesOf true "2" "rbcL" ["DTDILAAFR", 2;"FLFVAEAIYK", 2] 
    |> Array.ofSeq
let rbcsQuantification = 
    extractAbsoluteAbundancesOf true "2" "RBCS" ["AFPDAYVR", 2;"LVAFDNQK", 2] 
    |> Array.ofSeq

let protAbundanceChart =
    [
        Chart.Column(rbclQuantification |> Seq.map (fun x -> x.Filename + "_rbcL"), rbclQuantification |> Seq.map (fun x -> x.MeanQuant))
        |> Chart.withYErrorStyle (Array = (rbclQuantification |> Seq.map (fun x -> x.StdevQuant)))
        |> Chart.withTraceInfo "rbcL"
        Chart.Column(rbcsQuantification |> Seq.map (fun x -> x.Filename + "_rbcS"), rbcsQuantification |> Seq.map (fun x -> x.MeanQuant))
        |> Chart.withYErrorStyle (Array = (rbcsQuantification |> Seq.map (fun x -> x.StdevQuant)))
        |> Chart.withTraceInfo "RBCS"
    ]
    |> Chart.combine
    |> Chart.withYAxisStyle "protein abundance [amol/cell]"


In [ ]:
protAbundanceChart

// without filtering. Compare both Charts.
let rbclQuantification' = 
    extractAbsoluteAbundancesOf false "2" "rbcL" ["DTDILAAFR", 2;"FLFVAEAIYK", 2] 
    |> Array.ofSeq
let rbcsQuantification' = 
    extractAbsoluteAbundancesOf false "2" "RBCS" ["AFPDAYVR", 2;"LVAFDNQK", 2] 
    |> Array.ofSeq

let protAbundanceChart' =
    [
        Chart.Column(rbclQuantification' |> Seq.map (fun x -> x.MeanQuant), rbclQuantification' |> Seq.map (fun x -> x.Filename + "_rbcL"))
        |> Chart.withYErrorStyle (Array = (rbclQuantification' |> Seq.map (fun x -> x.StdevQuant)))
        |> Chart.withTraceInfo "rbcL"
        Chart.Column(rbcsQuantification' |> Seq.map (fun x -> x.MeanQuant), rbcsQuantification' |> Seq.map (fun x -> x.Filename + "_rbcS"))
        |> Chart.withYErrorStyle (Array = (rbcsQuantification' |> Seq.map (fun x -> x.StdevQuant)))
        |> Chart.withTraceInfo "RBCS"
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.light
    |> Chart.withYAxisStyle "protein abundance [amol/cell]"


In [ ]:
protAbundanceChart'

Since we didn't change the amount of QProt given to the sample but the amount of protein loaded into our SDS-PAGE, we check the reliability of our experiments via comparing 
the chart above with a chart of the protein quantification per band. We remember that the bands were loaded with different amounts of the proteins seperated by the SDS-PAGE.

Uncomment everything below.
This is the formula from above. Now that we want to get the protein per band in mol, just add a respective calculation to get the protein band in mol and define it in the 
anonymous record type.


In [ ]:
// let calcAbsoluteAbundance μgChlorophPerMlCult cellCountPerMlCult μgChlorophPerμlSample μgProtPerμlSample μgQProtSpike μgloadedProtein molWeightQProt molWeightTargetProt ratio1415N =
//     let chlorophPerCell : float = μgChlorophPerMlCult / cellCountPerMlCult
//     let cellsPerμlSample = μgChlorophPerμlSample / chlorophPerCell
//     let μgProteinPerCell = μgProtPerμlSample / cellsPerμlSample
//     let molQProtSpike = μgQProtSpike * 10. ** -6. / molWeightQProt
//     let molProtPerBand = ratio1415N * molQProtSpike
//     let molProtIn1μgLoadedProt = molProtPerBand / μgloadedProtein
//     let gTargetProtIn1μgLoadedProt = molWeightTargetProt * molProtIn1μgLoadedProt
//     let molProteinPerCell = molProtIn1μgLoadedProt * μgProteinPerCell
//     let proteinsPerCell = molProteinPerCell * 6.022 * 10. ** 23.
//     let attoMolProteinPerCell = molProteinPerCell * (10. ** 18.)
//     let attoMolProteinPerBand = ??? // <--- write your calculation here
//     {|
//         MassTargetProteinInLoadedProtein    = gTargetProtIn1μgLoadedProt
//         ProteinsPerCell                     = proteinsPerCell
//         AttoMolProteinPerCell               = attoMolProteinPerCell
//         AttoMolProteinPerBand               = attoMolProteinPerBand
//     |}

// let absoluteBandAbundances = 
//     ratios
//     |> Frame.map (fun peptide fileName ratio -> 
//         try 
//             let μgChlorophPerMlCult     = getμgChlorophPerMlCult fileName
//             let cellCountPerMlCult      = getCellCountPerMlCult fileName
//             let μgChlorophPerμlSample   = getμgChlorophPerμlSample fileName
//             let μgProtPerμlSample       = getμgProtPerμlSample fileName
//             let μgQProtSpike            = get15N_PS_Amount fileName
//             let μgloadedProtein         = getLoadAmount fileName
//             let molWeightQProt          = PSMass
//             let molWeightTargetProt     = peptide.AverageProtGroupMass
//             let result = 
//                 calcAbsoluteAbundance
//                     μgChlorophPerMlCult  
//                     cellCountPerMlCult   
//                     μgChlorophPerμlSample
//                     μgProtPerμlSample    
//                     (float μgQProtSpike)         
//                     μgloadedProtein
//                     molWeightQProt       
//                     molWeightTargetProt
//                     ratio
//             result.AttoMolProteinPerBand
//         with :? System.FormatException -> nan
//     )

// let extractAbsoluteBandAbundancesOf prot peptidelist = 
//     absoluteBandAbundances
//     |> Frame.filterRows (fun k s -> k.Synonyms |> String.contains prot)
//     |> Frame.filterRows (fun k s -> 
//         peptidelist |> List.exists (fun (sequence,charge) -> sequence = k.StringSequence && charge = k.Charge)
//     )
//     |> Frame.getNumericCols 
//     // |> Series.filter (fun k s -> getExpressionLevel k = "")
//     |> Series.map (fun k v -> 
//         {|
//             Synonym    = k 
//             MeanQuant  = Stats.mean v
//             StdevQuant = Stats.stdDev v
//         |}
//     )
//     |> Series.values

// let rbclBandQuantification = 
//     extractAbsoluteBandAbundancesOf "rbcL" ["DTDILAAFR", 2;"FLFVAEAIYK", 2] 
//     |> Array.ofSeq
// let rbcsBandQuantification = 
//     extractAbsoluteBandAbundancesOf "RBCS" ["AFPDAYVR", 2;"LVAFDNQK", 2] 
//     |> Array.ofSeq

// let protAbundanceBandChart =
//     [
//         Chart.Column(rbclBandQuantification |> Seq.map (fun x -> x.MeanQuant), rbclBandQuantification |> Seq.map (fun x -> x.Synonym))
//         |> Chart.withYErrorStyle (Array = (rbclBandQuantification |> Seq.map (fun x -> x.StdevQuant)))
//         |> Chart.withTraceInfo "rbcL"
//         Chart.Column(rbcsBandQuantification |> Seq.map (fun x -> x.MeanQuant), rbcsBandQuantification |> Seq.map (fun x -> x.Synonym))
//         |> Chart.withYErrorStyle (Array = (rbcsBandQuantification |> Seq.map (fun x -> x.StdevQuant)))
//         |> Chart.withTraceInfo "RBCS"
//     ]
//     |> Chart.combine
//     |> Chart.withTemplate ChartTemplates.light
//     |> Chart.withYAxisStyle "protein abundance per band [amol/band]"

// protAbundanceBandChart


Comparing this to the published results (see: https://www.frontiersin.org/articles/10.3389/fpls.2020.00868/full) we see that our preliminary results are
not only in the same order of magnitude as the published values, but in many cases really close! Of course it could be that you see systematic differences between your results
and published results. As data analysts it is now your task to estimate if the differences are the product of biology (e.g. different growth conditions or genetic background)
or caused by technical artifacts (e.g. different amounts of spiked proteins, mistakes estimating a parameter like the cell count) which could be accounted for by developing
normalization strategies. We look forward to read your explanations!
